In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
!pip install -U sentence-transformers

     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 450kB 10.4MB/s 
     |████████████████████████████████| 1.0MB 11.0MB/s 
     |████████████████████████████████| 870kB 51.0MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.5.1-cp36-none-any.whl size=67076 sha256=5dfde1813224179686f6d915eaad5b3cbcd5d8e588a457ea282dfcd6397d37bf
  Stored in directory: /root/.cache/pip/wheels/22/ca/b4/7ca542b411730a8840f8e090df2ddacffa1c4dd9f209684c19
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=9f3f674b880657c2a5dada5dc28c513f38d804942fde5535068e5875bcdb5245
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sentence-transformers sacremoses


In [0]:
from sentence_transformers import SentenceTransformer

In [7]:
!python --version

Python 3.6.9


In [0]:

import pandas
import numpy as np
import os
import pickle

In [9]:
model = SentenceTransformer('bert-base-nli-stsb-mean-tokens')

100%|██████████| 405M/405M [00:49<00:00, 8.19MB/s]


In [0]:
train_input = pandas.read_csv('./drive/My Drive/blosher_brar/blosher/train.csv')
test_input = pandas.read_csv('./drive/My Drive/blosher_brar/blosher/test.csv')
ref_input = pandas.read_csv('./drive/My Drive/blosher_brar/blosher/reference.csv')
sample_csv = pandas.read_csv('./drive/My Drive/blosher_brar/blosher/sample.csv')
text_input = pandas.read_csv('./drive/My Drive/blosher_brar/blosher/text_csv/text.csv')


In [19]:
text_input.head(5)

,id,title
0,0,interactive visual exploration of neighbor bas...
1,1,autodomainmine a graphical data mining system ...
2,2,anipqo almost non intrusive parametric query o...
3,3,relational division four algorithms and their ...
4,4,selection and ranking of text from highly impe...


In [20]:
text_input.dropna(axis=0, how='any', inplace=True)
text_input.loc[text_input.id.isnull(),:]

,id,title


In [21]:
train_input.head(5)

,id,label
0,0,1
1,3,1
2,6,1
3,8,0
4,9,0


In [22]:
train_input.dropna(axis=0, how='any', inplace=True)
train_input.loc[train_input.id.isnull(),:]

,id,label


In [23]:
test_input.head(5)

,id
0,1
1,2
2,4
3,5
4,7


In [24]:
test_input.dropna(axis=0, how='any', inplace=True)
test_input.loc[test_input.id.isnull(),:]

,id


In [25]:
ref_input.head(5)

,id,id.1
0,0,22305
1,0,22491
2,1,9243
3,1,10943
4,1,14322


In [26]:
ref_input.dropna(axis=0, how='any', inplace=True)
ref_input.loc[ref_input.id.isnull(),:]

,id,id.1


In [27]:
#This is for final output format only
sample_csv.head(5)

,id,label
0,1,2
1,2,0
2,4,3
3,5,3
4,7,4


In [0]:
ref_input.rename(columns={'id.1':'refid'}, inplace=True)

In [0]:
filename = './drive/My Drive/blosher_brar/blosher/text_csv/preprocessed.pkl'
if os.path.exists(filename):
    with open(filename, 'rb') as file:
        text_input = pickle.load(file)
    file.close()
else:
    ref_data_cleaned=ref_input.groupby(ref_input.id).refid.apply( lambda x: (' '.join(text_input.title[text_input.id.isin(x.tolist()) ].tolist()) ))  
    ref_data_cleaned_ids = ref_input.id.unique().tolist()
    text_input.loc[ ~text_input['id'].isin(ref_data_cleaned_ids), 'ref_data' ] = 'NoRef' 
    text_input.loc[ text_input['id'].isin(ref_data_cleaned_ids), 'ref_data' ] = ref_data_cleaned.tolist()
    text_input['title_n_ref'] = text_input['title'] + text_input['ref_data']
    text_input['bertvectors_data']=model.encode(text_input.title.tolist())
    ref_bertvectors_data = model.encode(text_input.ref_data.tolist())
    text_input['ref_bertvectors_data']= ref_bertvectors_data
    with open(filename, 'wb') as file:
        pickle.dump( text_input, file, protocol=pickle.HIGHEST_PROTOCOL)
    file.close()

In [39]:
text_input

,id,title,ref_data,title_n_ref,bertvectors_data,ref_bertvectors_data
0,0,interactive visual exploration of neighbor bas...,a framework for clustering evolving data strea...,interactive visual exploration of neighbor bas...,"[-0.05183797, -0.14493023, 0.5170056, 0.329029...","[-1.0956175, 0.22900814, 0.5946247, 0.18764888..."
1,1,autodomainmine a graphical data mining system ...,what is the nearest neighbor in high dimension...,autodomainmine a graphical data mining system ...,"[-0.6985498, 0.31020066, 0.572053, 0.23865114,...","[-0.9755858, 0.55166465, 0.9321183, 0.05515799..."
2,2,anipqo almost non intrusive parametric query o...,parametric query optimization optimization of ...,anipqo almost non intrusive parametric query o...,"[-1.3641545, -0.35146564, 0.3398267, 0.2292959...","[-1.327799, -0.11918369, 0.90907514, 0.5485083..."
3,3,relational division four algorithms and their ...,implementation techniques for main memory data...,relational division four algorithms and their ...,"[-0.2810368, 0.9038656, 0.7913686, -0.04569501...","[-0.6529086, 0.7173542, 0.31133357, 0.0052859,..."
4,4,selection and ranking of text from highly impe...,NoRef,selection and ranking of text from highly impe...,"[-0.7558338, 0.3391842, -0.026238041, 0.674918...","[-0.39454186, 0.7675157, 1.2327397, -0.3590787..."
...,...,...,...,...,...,...
25556,25556,dynamic typing in a statically typed language,quasi static typing an ideal model for recursi...,dynamic typing in a statically typed languageq...,"[-0.7299007, -0.72059864, 0.33246648, 0.181441...","[-1.1833999, -0.43209428, 0.73109716, 0.651279..."
25557,25557,maintaining materialized views in distributed ...,database snapshots maintenance of views the de...,maintaining materialized views in distributed ...,"[0.30573267, -0.1626336, 0.6847109, 0.32745653...","[-0.23411922, 0.43248, 1.2306722, 0.43173334, ..."
25558,25558,learning sparse metrics via linear programming,fastmap a fast algorithm for indexing data min...,learning sparse metrics via linear programming...,"[-0.93202716, -0.08457238, 0.0037817557, 0.639...","[-0.91630596, 0.19382447, 0.2923506, 0.4902326..."
25559,25559,computer assisted reasoning with mizar,NoRef,computer assisted reasoning with mizarNoRef,"[-1.3782543, 0.20787, 0.018869117, 0.04354784,...","[-0.39454186, 0.7675157, 1.2327397, -0.3590787..."


In [40]:
text_input.set_index('id', drop=False, inplace=True)
train_input.set_index('id', drop=False, inplace=True)
ref_input.set_index('id', drop=False, inplace=True)
test_input.set_index('id', drop=False, inplace=True)
print ()

In [41]:
inp_shape =  (-1, 4, 768//2)
all_data_list = []
for id_val in train_input.id.tolist():
    temp = text_input.bertvectors_data[id_val].tolist()
    temp.extend(text_input.ref_bertvectors_data[id_val])
    all_data_list.append(temp)
    
X_train = np.array(all_data_list)
print (X_train.shape)
X_train = X_train.reshape(inp_shape)
print (X_train.shape)
#X_train[0, 1, :]

all_data_list = []
for id_val in test_input.id.tolist():
    temp = text_input.bertvectors_data[id_val].tolist()
    temp.extend(text_input.ref_bertvectors_data[id_val])
    all_data_list.append(temp)
    
X_test = np.array(all_data_list)
print (X_test.shape)
X_test = X_test.reshape( inp_shape)
print (X_test.shape)
#X_train[0, 1, :]

(12779, 1536)
(12779, 4, 384)
(12782, 1536)
(12782, 4, 384)


In [42]:
Y_train = np.array(train_input.loc[train_input.id.tolist(), 'label'].tolist())
print (Y_train.shape)

(12779,)


In [0]:
from tensorflow import concat
from tensorflow.keras import Model, layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, RNN, Dense, LeakyReLU, SpatialDropout1D, Attention, Conv1D, SpatialDropout1D, GlobalAveragePooling1D ,MaxPooling1D
from tensorflow.keras.layers import BatchNormalization, Dropout, Embedding, Flatten, Input, Softmax, GRU, Reshape, Activation
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.metrics import Accuracy
import tensorflow.keras.backend as K
from tensorflow.keras import initializers, regularizers
from tensorflow.keras import optimizers

from tensorflow.keras.layers import Layer 
from tensorflow.keras import constraints


In [0]:
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    follows these equations:

    (1) u_t = tanh(W h_t + b)
    (2) \alpha_t = \frac{exp(u^T u)}{\sum_t(exp(u_t^T u))}, this is the attention weight
    (3) v_t = \alpha_t * h_t, v in time t
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        3D tensor with shape: `(samples, steps, features)`.
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape = (input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),#
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape = (input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight(shape = (input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero and this results in NaN's. 
        # Should add a small epsilon as the workaround
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a

        return weighted_input

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[1], input_shape[2]

class Addition(Layer):
    """
    This layer is supposed to add of all activation weight.
    We split this from AttentionWithContext to help us getting the activation weights
    follows this equation:
    (1) v = \sum_t(\alpha_t * h_t)

    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    """

    def __init__(self, **kwargs):
        super(Addition, self).__init__(**kwargs)

    def build(self, input_shape):
        self.output_dim = input_shape[-1]
        super(Addition, self).build(input_shape)

    def call(self, x):
        return K.sum(x, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [0]:
def Cascade_model():
    inp = Input(shape=X_train.shape[-2:])
    #print (inp.shape)
    print (inp[: ,:,:].shape)
    bi = Bidirectional(LSTM(units=256, return_sequences=True))
    cnn_layer1 = layers.Conv1D(filters=512,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
    cnn_layer2 = layers.Conv1D(filters=512,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
    cnn_layer3 = layers.Conv1D(filters=128,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
    pool = layers.GlobalMaxPool1D()
    dense_1 = layers.Dense(units=512)
    last_dense = layers.Dense(units=5, activation="softmax")
    
    #l_1 =bi(inp)
    l_1 = cnn_layer1(inp)
    l_1 = pool(l_1)
    
    l_2 = cnn_layer2(inp)
    l_2 = pool(l_2)
    
    l_4 =  cnn_layer3(inp)
    l_4 = pool(l_4)
    
    l_3 =bi(inp)
    l_3 = AttentionWithContext()(l_3)
    l_3 = Addition()(l_3)
    

    s_3= LSTM(units=512)(inp[: ,2:,:])
    #s_3=  BatchNormalization()(s_3)
    s_3= LeakyReLU()(s_3)
    
    s_4= Bidirectional(LSTM(units=256))(inp[: ,:2,:])
    #s_4=  BatchNormalization()(s_4)
    s_4= LeakyReLU()(s_4)
    
    
    
    concatenated = concat([l_1, l_2, l_3, l_4], axis=-1) 
    
    modell = dense_1(concatenated)
    model1 = LeakyReLU()(modell)
    modell = concat([modell, s_4, s_3], axis=-1)
    model = LeakyReLU()(modell)
    
    model = Dense(256)(model)
    model = LeakyReLU()(model)
    
    model = last_dense(model)
    model = Model( inp, model)
    
    model.summary()
    return model

In [103]:
cas_model = Cascade_model()    

(None, 4, 384)
Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(None, 4, 384)]     0                                            
__________________________________________________________________________________________________
bidirectional_33 (Bidirectional (None, 4, 512)       1312768     input_18[0][0]                   
__________________________________________________________________________________________________
conv1d_51 (Conv1D)              (None, 3, 512)       393728      input_18[0][0]                   
__________________________________________________________________________________________________
conv1d_52 (Conv1D)              (None, 3, 512)       393728      input_18[0][0]                   
____________________________________________________________________________

In [104]:
cas_model.compile(loss=SparseCategoricalCrossentropy(from_logits=True),optimizer=Adam(0.0001),metrics=["accuracy"])
cas_model.fit(x= X_train, y=Y_train,
              batch_size=128,
              validation_split=0.1,
              epochs=15,
              workers=-1,
              use_multiprocessing=True
             )

Epoch 1/15
90/90 [==============================] - 3s 33ms/step - loss: 1.3113 - accuracy: 0.6049 - val_loss: 1.1950 - val_accuracy: 0.7105
Epoch 2/15
90/90 [==============================] - 1s 16ms/step - loss: 1.1638 - accuracy: 0.7470 - val_loss: 1.1579 - val_accuracy: 0.7527
Epoch 3/15
90/90 [==============================] - 1s 16ms/step - loss: 1.1345 - accuracy: 0.7745 - val_loss: 1.1524 - val_accuracy: 0.7473
Epoch 4/15
90/90 [==============================] - 1s 16ms/step - loss: 1.1168 - accuracy: 0.7941 - val_loss: 1.1452 - val_accuracy: 0.7598
Epoch 5/15
90/90 [==============================] - 1s 16ms/step - loss: 1.1017 - accuracy: 0.8084 - val_loss: 1.1437 - val_accuracy: 0.7621
Epoch 6/15
90/90 [==============================] - 1s 16ms/step - loss: 1.0860 - accuracy: 0.8247 - val_loss: 1.1364 - val_accuracy: 0.7715
Epoch 7/15
90/90 [==============================] - 1s 17ms/step - loss: 1.0720 - accuracy: 0.8405 - val_loss: 1.1399 - val_accuracy: 0.7676
Epoch 8/15
90

In [105]:
X_test[:5].shape

(5, 4, 384)

In [0]:
predictions = cas_model.predict(X_test)

In [0]:
predictions_list = np.round(np.argmax(predictions, axis=1))

In [108]:
print (predictions_list.shape)
predictions_list

(12782,)


array([1, 1, 0, ..., 2, 2, 0])

In [0]:
prediction_output = pandas.DataFrame({'id': test_input.id.tolist() , 'label':list(predictions_list)} )

In [110]:
prediction_output

,id,label
0,1,1
1,2,1
2,4,0
3,5,2
4,7,2
...,...,...
12777,25553,1
12778,25556,3
12779,25558,2
12780,25559,2


In [0]:
prediction_output.to_csv('./drive/My Drive/blosher_brar/blosher/prediction_output.csv', index=False)